In [ ]:
!pip install gensim

In [ ]:
import tensorflow_text as text_tf
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from gensim.models import Word2Vec
import gensim
import numpy as np
import pandas as pd

In [ ]:
txt_path = "tausug_general_corpuz.txt"

with open(txt_path, "r", encoding="utf-8") as f:
  corpus = [ln.strip() for ln in f if ln.strip()]


This is good to catch unhelpful punctuations from cleaning. If next character is other than alphabets, new token.

In [ ]:
tokenizer = text_tf.UnicodeScriptTokenizer()

tokens = tokenizer.tokenize(corpus)

Convert the above byte strings to actual strings

In [ ]:
tokens_list = [
    [t.decode('utf-8') for t in tensor.numpy()]
    for tensor in tokens
]

print(tokens_list)


# flattened_tokens = list(set([items for sublist in tokens_list for items in sublist]))

# df_tokens = pd.DataFrame(data=flattened_tokens, columns=['tokens'])

# df_tokens

[['na', ',', 'miyadtu', 'napatinda', 'inmanga', 'budjang', 'dupang', 'namī', 'lana'], ['pag', "'", 'ubus', 'yadtu', ',', 'nakaī', 'gmayan', 'inmanga', 'taumataud', ',', 'miyawn', 'kaniya', 'inhangpu', "'", 'tagduwa', 'mulid', 'niya', 'iban', 'sinkaibanan', 'taunakarungug', 'sinnasī', 'hatniya'], ['iban', 'bukun', 'sadja', 'isab', 'yan', 'inmagpakita', "'", 'bang', 'biya', "'", 'diin', 'inpagpī', "'", 'sintuhan', 'ha', 'manga', 'tau'], ['iban', 'kiyabayta', "'", 'daisab', ',', 'amu', 'agi', ',', 'inmanga', 'kamu', 'bukun', 'bangsa', 'yahudi', 'pagkū', 'g', '-', 'kuyag', 'kamu', 'iban', 'sinmanga', 'taunapī', "'", 'suku', "'", 'sintuhan'], ['iban', 'laung', 'niya', 'paisab', 'kāku', "',", 'ayaw', 'mupagtapukan', 'inmanga', 'bayta', "',", 'amu', 'inparman', 'naug', 'dayn', 'hatuhan', ',', 'kiyabutang', 'halawm', 'sulat', 'yan', ',', 'sabab', 'masuuk', 'na', 'dumatung', 'inwaktu', 'maawn', 'inmanga', 'pakaradjaan', 'yan'], ['iban', 'laung', 'paisab', 'hiīsakanila', ',', 'hawnu', '-', 'hawn

Embedding Layer

In [ ]:
model = Word2Vec(tokens_list, min_count = 1, vector_size = 99, window = 5)

13252 sentences

In [ ]:
print(len(tokens_list))

13252


Capture the indices of the embedding layer for one-hot encoding

In [ ]:
embeddings = []
indices = []
for index, word in enumerate(model.wv):
    if index == len(model.wv) - 1:
      indices.append(index)
      embeddings.append(word)
      break;
    embeddings.append(word)
    indices.append(index)

embeddings = np.array(embeddings)


In [ ]:
len(indices)
vocab_size = len(embeddings)

One hot encoding. Must be float32 and a squared matrix

In [ ]:
one_hot_indices = tf.one_hot(indices=indices, depth=vocab_size, axis=1, dtype="float32")

In [ ]:
one_hot_indices

<tf.Tensor: shape=(19753, 19753), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [ ]:
embeddings.dtype

dtype('float32')

The LSTM Architecture

In [ ]:
lstm = keras.Sequential([keras.layers.LSTM(128), keras.layers.Dense(vocab_size, activation="softmax")])

Optimizers and Loss function that need to be idealized

In [ ]:
lstm.compile(optimizer = keras.optimizers.Adam(), loss=keras.losses.CategoricalCrossentropy(), metrics=[keras.metrics.Accuracy()])

Capture the mapped embedding of each word in a sentence with PADDING because numpy arrays need homogenous shape and take the last word as the y_train

In [ ]:
x_train = []
y_train = []
temp_x = []

for x in tokens_list:
  for word in x:
    if word == x[-1]:
      index = (model.wv.key_to_index[word])
      y_train.append(one_hot_indices[index])
      break
    temp_x.append(model.wv[word])

  x_train.append(temp_x)
  temp_x = []

In [ ]:
x_train = np.array(pad_sequences(x_train, padding='post'))
y_train = np.array(y_train)



13252/13252 ━━━━━━━━━━━━━━━━━━━━ 700s 53ms/step - accuracy: 0.0000e+00 - loss: 7.0496


Fitting the model

In [ ]:
fitted_lstm = lstm.fit(x=x_train, y=y_train, batch_size=1,  epochs=1)

In [ ]:
lstm.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_12 (LSTM)                  │ (1, 128)               │       116,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (1, 19753)             │     2,548,137 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,994,621 (30.50 MB)

 Trainable params: 2,664,873 (10.17 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,329,748 (20.33 MB)

In [ ]:
output = lstm.predict(x_train)
print(output)

415/415 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step
[[4.6218259e-04 8.6932761e-07 3.3916607e-02 ... 8.4847812e-07
  8.5959630e-07 8.7108867e-07]
 [4.6494999e-04 8.6805414e-07 3.3939898e-02 ... 8.4715117e-07
  8.5828225e-07 8.6979594e-07]
 [4.6317521e-04 8.6793563e-07 3.3915333e-02 ... 8.4707023e-07
  8.5820187e-07 8.6968879e-07]
 ...
 [4.6290262e-04 8.6795524e-07 3.3900481e-02 ... 8.4708529e-07
  8.5822614e-07 8.6970761e-07]
 [4.6084906e-04 8.6846387e-07 3.3922840e-02 ... 8.4764235e-07
  8.5875291e-07 8.7023392e-07]
 [4.6135121e-04 8.6794307e-07 3.3903401e-02 ... 8.4709200e-07
  8.5822558e-07 8.6970624e-07]]


In [ ]:
model.save('lstm_simple_architecture.keras')